Import một số thư viện

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from gensim.models import Word2Vec

Kết nối xuống driver

In [ ]:
#Connect to gg drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/Colab Notebooks/dataset")

Mounted at /content/drive


Tải trained glove và dataset

In [ ]:
raw_data = pd.read_csv('new_edited_LSTM_data.csv')
raw_data.clean_text = raw_data.clean_text.astype(str)

In [ ]:
raw_data

,text,sentiment,rate,tokens,clean_text
0,The camera is very slow :(,neutral,3.0,"['camera', 'slow']",camera slow
1,I was happy I got a case and a charger that wa...,negative,1.0,"['happy', 'got', 'case', 'charger', 'descripti...",happy got case charger description told phone ...
2,I bought this phone for my wife and I wanted t...,positive,5.0,"['bought', 'phone', 'wife', 'wanted', 'wait', ...",bought phone wife wanted wait month half leave...
3,HTC One in White is good; in black it is bette...,positive,5.0,"['tc', 'one', 'white', 'good', 'black', 'bette...",tc one white good black better love new phone ...
4,This phone is definitely one of the fastest on...,positive,5.0,"['phone', 'definitely', 'one', 'fastest', 'mar...",phone definitely one fastest market moment dro...
...,...,...,...,...,...
89995,Fast but hot. Which makes it run slower in hig...,neutral,3.0,"['fast', 'hot', 'makes', 'run', 'slower', 'hig...",fast hot makes run slower high temperature sit...
89996,no devise help,neutral,3.0,"['no', 'devise', 'help']",no devise help
89997,I love the watch/phone and everything it offer...,neutral,3.0,"['love', 'watch', 'phone', 'everything', 'offe...",love watch phone everything offered keep freez...
89998,Replacement,neutral,3.0,['replacement'],replacement


In [ ]:
data = raw_data

In [ ]:
model_word2vec = Word2Vec.load("Word2Vec_model.w2v")

In [ ]:
model_word2vec.wv["the"].shape[0]

300

Thực hiện Tokenize đoạn văn bản

In [ ]:
vocab_size = 15000

tokenizer = Tokenizer(split=' ', num_words = vocab_size)
tokenizer.fit_on_texts(data['clean_text'].values)
print()
X = tokenizer.texts_to_sequences(data['clean_text'].values) #convert data to number
X = pad_sequences(X) # padding
print(X.shape[1]) #check size
MAX_LEN = X.shape[1]
words_to_index = tokenizer.word_index





1607


In [ ]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

top_n_words_index = dict(take(vocab_size, words_to_index.items()))
len(top_n_words_index)

15000

Khởi tạo model LSTM

In [ ]:
inp_dim = vocab_size+1
embed_dim = model_word2vec.wv['the'].shape[0]
hidden_nodes = 150 #dimension of LSTM output
output_dim = 3

print(vocab_size)
print(inp_dim)

15000
15001


In [ ]:
# Tạo matrix của trained glove 
emb_matrix = np.zeros((inp_dim, embed_dim))
for word, index in top_n_words_index.items():
  try:
    embedding_vector = model_word2vec.wv[word]
  except:
    continue
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

In [ ]:
model = Sequential()
#Embedding each row of X ("X.shape[1]" len) into dense vectors "embed_dim"
# model.add(Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length = MAX_LEN)) 
model.add(Embedding(input_dim = inp_dim, output_dim = embed_dim, input_length = MAX_LEN, weights = [emb_matrix], trainable=False)) 
model.add(SpatialDropout1D(0.4))
model.add(LSTM(hidden_nodes, dropout=0.2, recurrent_dropout=0.2))  # add return_sequences=True if you want many-to-many LSTM model
model.add(Dense(output_dim,activation='softmax')) 
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1607, 300)         4500300   
                                                                 
 spatial_dropout1d (SpatialD  (None, 1607, 300)        0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 150)               270600    
                                                                 
 dense (Dense)               (None, 3)                 453       
                                                                 
Total params: 4,771,353
Trainable params: 271,053
Non-trainable params: 4,500,300
_________________________________________________________________
None


Tạo tập train và test

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(72000, 1607) (72000, 3)
(18000, 1607) (18000, 3)


In [ ]:
Y_train

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       ...,
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1]], dtype=uint8)

Thực hiện train network 

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 4, batch_size=batch_size, verbose = 1)

Epoch 1/4
2250/2250 [==============================] - 14408s 6s/step - loss: 0.6569 - accuracy: 0.7072
Epoch 2/4
2250/2250 [==============================] - 14342s 6s/step - loss: 0.5664 - accuracy: 0.7590
Epoch 3/4
2250/2250 [==============================] - 14450s 6s/step - loss: 0.5383 - accuracy: 0.7714
Epoch 4/4
2250/2250 [==============================] - 14685s 7s/step - loss: 0.5233 - accuracy: 0.7806


Lưu model xuống drive

In [ ]:
#Lưu model
model.save('LSTM_Model.h5')

In [ ]:
#Lưu train và test
from numpy import savetxt
savetxt('X_train.csv', X_train, delimiter=',')
savetxt('Y_train.csv', Y_train, delimiter=',')
savetxt('X_test.csv', X_test, delimiter=',')
savetxt('Y_test.csv', Y_test, delimiter=',')

In [ ]:
#Lưu tokenizer
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('MAX_LEN.txt', 'w') as f:
  f.write('%d' % MAX_LEN)

Test sử dụng LSTM

In [ ]:
twt = ["""product bad never buy"""]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=MAX_LEN, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
np.argmax(sentiment)
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("neutral")
elif(np.argmax(sentiment) == 2):
  print("positive")

[[ 0  0  0 ... 57 52 23]]
1/1 - 2s - 2s/epoch - 2s/step
negative


In [ ]:
type(twt)

numpy.ndarray

Validation và evaluation

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

Dropout: 20% - https://towardsdatascience.com/choosing-the-right-hyperparameters-for-a-simple-lstm-using-keras-f8e9ed76f046


Tham khảo: https://colab.research.google.com/drive/1fAuZ7L8YQZrF-MyRXrdyR-Dh13AfA_X0

https://towardsdatascience.com/sentiment-analysis-using-lstm-and-glove-embeddings-99223a87fe8e